In [1]:
#load data and packages
import pandas as pd
import numpy as np
import json as json
import matplotlib.pyplot as plt 



import matplotlib.pyplot as plt
import seaborn as sns

import keras
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras import losses
from keras import optimizers
from keras import metrics
from sklearn.metrics import classification_report,confusion_matrix
from sklearn import metrics
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D,MaxPool2D
from keras.layers.normalization import BatchNormalization
import numpy as np

import tensorflow as tf

import cv2
import os

from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
from sklearn import metrics
import itertools

with open('/Users/mattvalko/Documents/PHD/metadata.json',encoding="latin-1") as json_data:
    data = json.load(json_data)

Using TensorFlow backend.


In [2]:
# Load in the background data /file directory
info = pd.json_normalize(data["annotations"])
label = pd.json_normalize(data["categories"])
files = pd.json_normalize(data[ "images"])
files['folder']=files['file_name'].str[7:10]
fullDF = files[files["folder"].astype(int) < 11]
merged=info.merge(fullDF,on='id',how="inner")
cnts=merged.groupby(['category_id'])['id'].count()
catIDS=cnts[cnts > 700]#.index
final = merged[merged['category_id'].isin(catIDS.index)]
imgDir=final['file_name'].str[7::]
imgDir=np.array(imgDir)
path='/Users/mattvalko/Documents/PHD/examdata/'

In [3]:
#Define the label for classification
label=np.array(final['category_id'])
u, y_train = np.unique(label, return_inverse=True)
classNum=len(catIDS)
y_train = keras.utils.to_categorical(y_train, classNum)

In [4]:
classNum

2

In [5]:
# load data 
data = [] 
img_height,img_width = 100,100 
for i in range(0,len(imgDir)):
    img_arr = cv2.imread(os.path.join(path, imgDir[i]))
    resized_arr = cv2.resize(img_arr, (img_height,img_width))
    data.append(resized_arr)

In [9]:

# Normalize the data
x_feat=np.array(data)
x_train = x_feat / 255


x_train.reshape(-1,img_height,img_width, 1)

array([[[[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        ...,

        [[0.4745098 ],
         [0.52941176],
         [0.41568627],
         ...,
         [0.49411765],
         [0.36078431],
         [0.53333333]],

        [[0.63137255],
         [0.42745098],
         [0.57254902],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.95294118],
         [0.95294118],
         [0.91764706]]],


       [[[0.94117647],
         [0.94901961],
         [0.96

In [8]:
#Load Keras packages
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import os
from urllib.request import urlopen,urlretrieve
from PIL import Image
from tqdm import tqdm_notebook
from sklearn.utils import shuffle
import cv2

from keras.models import load_model
from sklearn.datasets import load_files   
from keras.utils import np_utils
from glob import glob
from keras import applications
from keras.preprocessing.image import ImageDataGenerator 
from keras import optimizers
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.callbacks import TensorBoard,ReduceLROnPlateau,ModelCheckpoint
num_classes = classNum
from keras.optimizers import SGD, Adam


In [6]:

#If imagenet weights are being loaded, 
#input must have a static square shape (one of (128, 128), (160, 160), (192, 192), or (224, 224))
base_model = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape= (img_height,img_width,3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(num_classes, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

/Users/mattvalko/anaconda3/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [7]:

from keras.optimizers import SGD, Adam
# sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
adam = Adam(lr=0.0001)
model.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])

array([[[[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        ...,

        [[0.4745098 ],
         [0.52941176],
         [0.41568627],
         ...,
         [0.49411765],
         [0.36078431],
         [0.53333333]],

        [[0.63137255],
         [0.42745098],
         [0.57254902],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.95294118],
         [0.95294118],
         [0.91764706]]],


       [[[0.94117647],
         [0.94901961],
         [0.96

In [9]:
resnet50 = model.fit(x_train, y_train,
epochs=5,
verbose=1)

Epoch 1/5
2001/2001 [==============================] - 20784s 10s/step - loss: 0.5024 - accuracy: 0.8351
Epoch 2/5
2001/2001 [==============================] - 17998s 9s/step - loss: 0.1350 - accuracy: 0.9675
Epoch 3/5
2001/2001 [==============================] - 3160s 2s/step - loss: 0.0705 - accuracy: 0.9800
Epoch 4/5
2001/2001 [==============================] - 343s 171ms/step - loss: 0.0726 - accuracy: 0.9815
Epoch 5/5
2001/2001 [==============================] - 315s 157ms/step - loss: 0.0422 - accuracy: 0.9890


In [ ]:
#Resnet101
base_model = applications.resnet.ResNet101(weights= None, include_top=False, input_shape= (img_height,img_width,3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(num_classes, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)


In [ ]:
adam = Adam(lr=0.0001)
model.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])
resnet101 = model.fit(x_train, y_train,
epochs=5,
verbose=1)

In [ ]:
model.summary()


In [ ]:
base_model = applications.Xception(weights= None, include_top=False, input_shape= (img_height,img_width,3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(num_classes, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

adam = Adam(lr=0.0001)
model.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])
xception101 = model.fit(x_train, y_train,
epochs=1,
verbose=1, batch_size=100)


In [11]:
base_model = applications.VGG16(weights= None, include_top=False, input_shape= (img_height,img_width,3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(num_classes, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

adam = Adam(lr=0.0001)
model.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])
VGG16 = model.fit(x_train, y_train,
epochs=1,
verbose=1)

Epoch 1/1
2001/2001 [==============================] - 300s 150ms/step - loss: 0.5799 - accuracy: 0.6437


In [11]:
base_model = applications.VGG19(weights= None, include_top=False, input_shape= (img_height,img_width,3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(num_classes, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

adam = Adam(lr=0.0001)
model.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])
VGG19 = model.fit(x_train, y_train,
epochs=1,
batch_size = 32
,verbose=1)

Epoch 1/1
2001/2001 [==============================] - 361s 180ms/step - loss: 0.6318 - accuracy: 0.5947


In [ ]:
base_model = applications.InceptionV3(weights= None, include_top=False, input_shape= (img_height,img_width,3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(num_classes, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

adam = Adam(lr=0.0001)
model.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])
inceptionV3 = model.fit(x_train, y_train,
epochs=1,
batch_size = 32
,verbose=1)

In [ ]:
acc = history.history['accuracy']
#val_acc = history.history['val_accuracy']
loss = history.history['loss']

epochs_range = range(5)

plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
y_pred=model.predict_classes(xtest)
y_true=np.argmax(ytest,axis=1)

In [ ]:
opt = Adam(lr=0.001)
AlexNet.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])


In [ ]:
Alexnethistory = AlexNet.fit(xtrain, ytrain,
epochs=5,
verbose=1,
validation_data=(xtest, ytest))

In [ ]:
base_model = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape= (img_height,img_width,3))

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import os
from urllib.request import urlopen,urlretrieve
from PIL import Image
from tqdm import tqdm_notebook
from sklearn.utils import shuffle
import cv2

from keras.models import load_model
from sklearn.datasets import load_files   
from keras.utils import np_utils
from glob import glob
from keras import applications
from keras.preprocessing.image import ImageDataGenerator 
from keras import optimizers
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.callbacks import TensorBoard,ReduceLROnPlateau,ModelCheckpoint

In [ ]:

img_height,img_width = 64,64 
num_classes = 6
#If imagenet weights are being loaded, 
#input must have a static square shape (one of (128, 128), (160, 160), (192, 192), or (224, 224))
base_model = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape= (img_height,img_width,3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(num_classes, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

In [ ]:

from keras.optimizers import SGD, Adam
# sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
adam = Adam(lr=0.0001)
model.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
https://towardsdatascience.com/understanding-and-coding-a-resnet-in-keras-446d7ff84d33